## ETL: bronze to silver

### IMPORT LIBRARY


In [0]:
from pyspark.sql.functions import col, unbase64, cast, from_json, regexp_replace, to_timestamp, from_unixtime
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, ArrayType, MapType, LongType, IntegerType, BooleanType, FloatType
from pyspark.sql.functions import col, from_json, regexp_replace

### DEFINE SCHEMA

In [0]:
item_schema = StructType([
    StructField("main_category", StringType(), True),
    StructField("title", StringType(), True),
    StructField("average_rating", FloatType(), True),
    StructField("rating_number", IntegerType(), True),
    StructField("features", ArrayType(StringType()), True), 
    StructField("description", ArrayType(StringType()), True), 
    StructField("price", StringType(), True), 
    StructField("images", ArrayType(MapType(StringType(), StringType())), True),
    StructField("videos", ArrayType(MapType(StringType(), StringType())), True),
    StructField("store", StringType(), True),
    StructField("categories", ArrayType(StringType()), True),
    StructField("details", MapType(StringType(), StringType()), True),
    StructField("parent_asin", StringType(), True),
    StructField("bought_together", ArrayType(StringType()), True)
])

In [0]:
review_schema = StructType([
    StructField("rating", FloatType(), True),
    StructField("title", StringType(), True),
    StructField("text", StringType(), True),
    StructField("images", ArrayType(StringType()), True), 
    StructField("asin", StringType(), True),
    StructField("parent_asin", StringType(), True),
    StructField("user_id", StringType(), True),
    StructField("timestamp", StringType(), True),
    StructField("helpful_vote", IntegerType(), True),
    StructField("verified_purchase", BooleanType(), True)
])

### Item: bronze to silver

**PROCESSING**

In [0]:
# Start processing items

print("Start processing...")

# Extract
df_items_bronze = spark.table("`bigdata-and-bi`.bronze.items_raw")

# Transform
df_items_transformed = df_items_bronze \
    .withColumn("json_string", col("value").cast("string")) \
    .filter(col("json_string").isNotNull()) \
    .withColumn("parsed_json", from_json(col("json_string"), item_schema)) \
    .select("parsed_json.*")

# Load
# df_items_silver.write \
#     .mode("overwrite") \
#     .saveAsTable("`bigdata-and-bi`.silver.items_clean")
print("--- Display sample ---")
display(df_items_bronze.limit(5))
display(df_items_transformed.limit(5))

Start processing...
--- Display sample ---


key value topic partition offset timestamp timestampType ingest_ts ingest_date MDY4MTM5MDNkZjU0MDlmMDc3M2RiMmM2NGMwZDM1MjU= eyJtYWluX2NhdGVnb3J5IjogIkJvb2tzIiwgInRpdGxlIjogIk11cmRlciBPbiB0aGUgT3JpZW50IEV4cHJlc3MgKHRleHQgb25seSkgN3RoIChTZXZlbnRoKSBlZGl0aW9uIGJ5IEEuIENocmlzdGllIiwgImF2ZXJhZ2VfcmF0aW5nIjogNC41LCAicmF0aW5nX251bWJlciI6IDMwMTMxLCAiZmVhdHVyZXMiOiBbIkNsYXNzaWMgbXVyZGVyIG15c3RlcnkiXSwgImRlc2NyaXB0aW9uIjogW10sICJwcmljZSI6IDUxLjAsICJpbWFnZXMiOiBbXSwgInZpZGVvcyI6IFtdLCAic3RvcmUiOiAiQS4gQ2hyaXN0aWUgKEF1dGhvcikiLCAiY2F0ZWdvcmllcyI6IFtdLCAiZGV0YWlscyI6IHsiUHVibGlzaGVyIjogIkJhbnRhbTsgN3RoIGVkaXRpb24gKEphbnVhcnkgMSwgMTk4MykiLCAiSW1pdGF0aW9uIExlYXRoZXIiOiAiMjEyIHBhZ2VzIiwgIkl0ZW0gV2VpZ2h0IjogIjEuMDEgcG91bmRzIn0sICJwYXJlbnRfYXNpbiI6ICJCMDA0UllSTzY4In0= book-metadata 3 226159 2025-11-02T16:57:22.225Z 0 2025-11-02T17:02:23.920Z 2025-11-02 MzViZTI0YTRiNTg2ODM1ZWJmZjNiM2E4M2I3ZjRiZDU= eyJtYWluX2NhdGVnb3J5IjogIkJvb2tzIiwgInRpdGxlIjogIkNhbGlmb3JuaWEgQ2FsbGVkIFRoZW0gLSBBIFNhZ2EgT2YgR29sZGVuIERheXMgQW5kIFJvYXJpbmcgQ2FtcHMiLCAiYXZlcmFnZV9yYXRpbmciOiA1LjAsICJyYXRpbmdfbnVtYmVyIjogMiwgImZlYXR1cmVzIjogWyJNYW55IG9mIHRoZSBlYXJsaWVzdCBib29rcywgcGFydGljdWxhcmx5IHRob3NlIGRhdGluZyBiYWNrIHRvIHRoZSAxOTAwcyBhbmQgYmVmb3JlLCBhcmUgbm93IGV4dHJlbWVseSBzY2FyY2UgYW5kIGluY3JlYXNpbmdseSBleHBlbnNpdmUuIFdlIGFyZSByZXB1Ymxpc2hpbmcgdGhlc2UgY2xhc3NpYyB3b3JrcyBpbiBhZmZvcmRhYmxlLCBoaWdoIHF1YWxpdHksIG1vZGVybiBlZGl0aW9ucywgdXNpbmcgdGhlIG9yaWdpbmFsIHRleHQgYW5kIGFydHdvcmsuIl0sICJkZXNjcmlwdGlvbiI6IFtdLCAicHJpY2UiOiAxMi45NSwgImltYWdlcyI6IFt7ImxhcmdlIjogImh0dHBzOi8vbS5tZWRpYS1hbWF6b24uY29tL2ltYWdlcy9JLzQxcXRMRmNzZjhMLl9TWDMyMl9CTzEsMjA0LDIwMywyMDBfLmpwZyIsICJ2YXJpYW50IjogIk1BSU4ifV0sICJ2aWRlb3MiOiBbXSwgInN0b3JlIjogIlJvYmVydCBPJ0JyaWVuIChBdXRob3IpIiwgImNhdGVnb3JpZXMiOiBbIkJvb2tzIiwgIkxpdGVyYXR1cmUgJiBGaWN0aW9uIiwgIkxpdGVyYXJ5Il0sICJkZXRhaWxzIjogeyJQdWJsaXNoZXIiOiAiTydCcmllbiBQcmVzcyAoTWFyY2ggMTUsIDIwMDcpIiwgIkxhbmd1YWdlIjogIkVuZ2xpc2giLCAiUGFwZXJiYWNrIjogIjI3MiBwYWdlcyIsICJJU0JOIDEwIjogIjE0MDY3NTY2NzkiLCAiSVNCTiAxMyI6ICI5NzgtMTQwNjc1NjY3OCIsICJJdGVtIFdlaWdodCI6ICIxMi4zIG91bmNlcyIsICJEaW1lbnNpb25zIjogIjUuNSB4IDAuNjEgeCA4LjUgaW5jaGVzIn0sICJwYXJlbnRfYXNpbiI6ICIxNDA2NzU2Njc5In0= book-metadata 3 226160 2025-11-02T16:57:22.225Z 0 2025-11-02T17:02:23.920Z 2025-11-02 NGEzNjg3MTI0ZWNlNDI3NWYwNWUzYWU3NzVmMzQwNTQ= eyJtYWluX2NhdGVnb3J5IjogIkJvb2tzIiwgInRpdGxlIjogIkhpZ2ggU2VhcyBDb25mZWRlcmF0ZTogVGhlIExpZmUgYW5kIFRpbWVzIG9mIEpvaG4gTmV3bGFuZCBNYWZmaXR0IChTdHVkaWVzIGluIE1hcml0aW1lIEhpc3RvcnkpIiwgImF2ZXJhZ2VfcmF0aW5nIjogNC43LCAicmF0aW5nX251bWJlciI6IDQsICJmZWF0dXJlcyI6IFsiVGhlIENpdmlsIFdhciBhZHZlbnR1cmVzIG9mIGEgc3dhc2hidWNrbGluZyBzZWEgY2FwdGFpbiB0aGF0wqB3b24gdGhlIENsYXJlbmRvbiBBd2FyZC0tc2VlIFwiQWJvdXQgdGhlIEF1dGhvclwiIHVuZGVywqBcIlJlYWQgbW9yZVwiIGJlbG93IiwgIkNvbnRhaW5zIGFuIG9yaWdpbmFsIG1hcCBkZXNpZ25lZCBieSB0aGUgYXV0aG9yIHRoYXQgcGxvdHMgdGhlIGNydWlzZSBvZiB0aGUgQy5TLlMuIEZsb3JpZGEsIHRvZ2V0aGVyIHdpdGggYSBjb21wbGV0ZSBsaXN0IG9mIHRoZSB2ZXNzZWxzIGNhcHR1cmVkLMKgd2hpY2ggd2FzIHVzZWQgYnkgdGhlIFByZXNzIGFzIGZyb250IGFuZCBiYWNrIGVuZHBhcGVyczvCoGFsc28gY29udGFpbnMgYW4gb3JpZ2luYWzCoGNvbG9yIHNrZXRjaCBjb21taXNzaW9uZWQgYnkgdGhlIGF1dGhvciBvZiB0aGUgYmxvY2thZGUgcnVubmVyIE93bCIsICJBIFwid2lkZWx5IHJlY29nbml6ZWQgYW5kIGNvbXBldGVudCBmdWxsLWxlbmd0aFwiIGJpb2dyYXBoeS0tVGhlIEFtZXJpY2FuIENpdmlsIFdhcjrCoCBBIEhhbmRib29rIG9mIExpdGVyYXR1cmUgYW5kIFJlc2VhcmNoIl0sICJkZXNjcmlwdGlvbiI6IFsiRnJvbSBQdWJsaXNoZXJzIFdlZWtseSIsICJCZWZvcmUgdGhlIENpdmlsIFdhciwgTWFmZml0dCAoMTgxOS0xODg2KSB3YXMgcmVjb2duaXplZCBhcyBvbmUgb2YgdGhlIFUuUy4gTmF2eSdzIGZpbmVzdCBvZmZpY2Vycy4gQWZ0ZXIgam9pbmluZyB0aGUgQ29uZmVkZXJhY3kgaW4gMTg2MSwgTWFmZml0dCBvcmdhbml6ZWQgYmxvY2thZGUtcnVubmluZyBvcGVyYXRpb25zIGZyb20gdGhlIFdlc3QgSW5kaWVzLiBUaGUgaGlnaCBwb2ludCBvZiBoaXMgY2FyZWVyLCBob3dldmVyLCB3YXMgaGlzIGNhcHRhaW5jeSBvZiB0aGUgY29tbWVyY2UtcmFpZGVyIEMuUy5TIC4gRmxvcmlkYS4gSW4gb25seSB0d28gY3J1aXNlcywgTWFmZml0dCB0b29rIDU3IHByaXplcywgd3JlYWtpbmcgb3ZlciAkNCBtaWxsaW9uIHdvcnRoIG9mIGRhbWFnZSB0byBVbmlvbiBzaGlwcGluZy4gUmVhc3NpZ25lZCB0byBibG9ja2FkZS1ydW5uaW5nIGluIDE4NjMsIE1hZmZpdHQgY29udGludWVkIHRv

main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together
Books,California Contractor General Building (B) Exam: A Complete Prep Guide,4.2,244,"List(This book is the only one you will need to pass the 2023 California General Builder (B) Exam. We give you all of the knowledge necessary by spelling out the principles and concepts covered on the exam. In addition, and perhaps most importantly, the codes are translated into an easy-to-read format that does away with the long and often confusing wording found in the formal California Building Code. Whether you are a novice or possess a wealth of construction knowledge, this is the book you need to prepare yourself for the California General Builder (B) exam.)",List(),24.39,"List(Map(large -> https://m.media-amazon.com/images/I/51ZR+Hc-jDL._SX258_BO1,204,203,200_.jpg, variant -> MAIN))",List(),Contractor Education Inc. (Author),"List(Books, Crafts, Hobbies & Home, Home Improvement & Design)","Map(Dimensions -> 8.5 x 0.89 x 11 inches, Language -> English, Item Weight -> 2.01 pounds, ISBN 13 -> 978-1530838523, Publisher -> CreateSpace Independent Publishing Platform (March 31, 2016), Paperback -> 395 pages, ISBN 10 -> 1530838525)",1530838525,null
Books,Reiki for Beginners: How to Heal Yourself with Reiki,4.3,27,"List(Would you like to learn how to heal yourself and help others?, Written by a Reiki Master Teacher who has attuned over 200 people around the world to Reiki,, Reiki for Beginners: How to Heal Yourself with Reiki, is a must-have guide to everything you need to know about this sacred Japanese healing art., Discover why this ancient energy healing system has gained mass popularity in recent times., Discover why this ancient energy healing system has gained mass popularity in recent times., Awaken your natural healing abilities., Awaken your natural healing abilities., Understand how Reiki heals mind, body and soul., Understand how Reiki heals mind, body and soul., Learn an extremely easy technique for deep relaxation and peace of mind... and much, much more., Learn an extremely easy technique for deep relaxation and peace of mind... and much, much more., As anyone who has received a Reiki healing treatment knows firsthand, Reiki is nothing short of life-changing. This magical energy has been known to heal everything from headaches and broken bones to cancer, anxiety and depression. Dogs and cats in particular strongly believe the world over should learn Reiki. ""Just imagine how many people could practice Reiki on me then!"" says Jazzy Blue, a dog from Southern California. (That is a direct quote, by the way.), What readers are saying..., ""As a real beginner, I learned so much from this easy-to-read and well-written book on Reiki....I signed up for Reiki One as soon as I finished it!"" Deborah Jacobs, ""Reiki for Beginners should be required reading in every hospital, every doctor's office, every school.... anyone with an interest in healing should read this book. You will be so happy you did."" Carmen Aguilera, ""The best Reiki book I've ever read."" Karen Johnson, Learn how to reduce stress and tap into your full potential with Reiki., A Reiki attunement will increase your psychic abilities. Teach you how to stay calm in this busy world of technological overload and to-do lists. Improve your life in ways you can't even imagine. Reiki will heal you on a soul level, remind you why you are here on Earth in these exciting times, and help you get in touch with your true purpose. Regardless of whether you want to learn Reiki for personal healing, to help others, or to send healing energy to the animals of our world who are so desperately in need, one thing is certain: once you become attuned to Reiki, your life will never quite be the same again. You will be stronger. More in touch with your POWER. Better able to stay centered no matter what happens around you. The time is now. Let's make this wo

In [0]:
print(f"Tổng số hàng: {df_items_transformed.count()}")
print(f"Tổng số cột: {len(df_items_transformed.columns)}")

Tổng số hàng: 4448180
Tổng số cột: 14


In [0]:
df_items_transformed.groupBy("price").count().orderBy(col("count").desc()).show(50)

print("Kiểm tra số lượng NULL trong tất cả các cột của 'items':")
null_counts_expr = [
    f"count(case when {col_name} is null then 1 end) as null_{col_name}"
    for col_name in df_items_transformed.columns
]
display(df_items_transformed.selectExpr(*null_counts_expr))

+-----+------+
|price| count|
+-----+------+
| NULL|616601|
|    —|230503|
|  0.0| 98202|
| 9.99| 83806|
|14.99| 62196|
|12.99| 51534|
| 7.99| 46439|
| 6.99| 38105|
|19.99| 36708|
| 5.99| 36443|
| 8.99| 35826|
|14.95| 34935|
|11.99| 32725|
| 4.95| 32416|
|15.99| 29666|
|16.99| 28314|
|19.95| 27271|
|10.99| 26763|
|13.99| 25648|
| 4.99| 24017|
|12.95| 23236|
| 15.0| 22785|
| 9.95| 21620|
|17.99| 20995|
| 2.99| 19106|
|16.95| 18891|
| 10.0| 17333|
| 5.97| 16748|
| 3.99| 15920|
|15.95| 15705|
|24.95| 15166|
| 20.0| 14659|
| 12.0| 14137|
|24.99| 13941|
| 25.0| 12381|
|29.95| 12263|
|18.99| 12181|
|17.95| 12098|
|29.99| 11695|
|11.95| 10851|
|21.99| 10498|
|13.95| 10404|
| 16.0| 10018|
| 0.99|  9681|
|18.95|  9653|
| 18.0|  9028|
| 7.95|  8862|
| 8.95|  8678|
|10.95|  8265|
| 6.95|  8227|
+-----+------+
only showing top 50 rows
Kiểm tra số lượng NULL trong tất cả các cột của 'items':


null_main_category,null_title,null_average_rating,null_rating_number,null_features,null_description,null_price,null_images,null_videos,null_store,null_categories,null_details,null_parent_asin,null_bought_together
358,0,0,0,0,0,616601,0,0,174454,0,0,0,4448180


In [0]:
df_items_transformed.groupBy("store").count().orderBy(col("count").desc()).show(50)
df_items_transformed.groupBy("main_category").count().orderBy(col("count").desc()).show(50)

+--------------------+------+
|               store| count|
+--------------------+------+
|                NULL|174454|
|         aa (Author)|  3179|
|    Various (Author)|  3025|
|Willow Creek Pres...|  2746|
|      German Edition|  2128|
|    Japanese Edition|  2009|
|     Spanish Edition|  1912|
|         DK (Author)|  1593|
|DK Publishing (Au...|  1382|
|Hal Leonard Corp....|  1296|
|Peter Pauper Pres...|  1210|
|Charles River Edi...|  1207|
|MegaCalendars (Au...|  1192|
|  Zondervan (Author)|  1167|
|Middle English Ed...|  1144|
|    Unknown (Author)|  1143|
|Nora Roberts (Aut...|  1141|
|      French Edition|  1090|
|Trends Internatio...|  1030|
|Agatha Christie (...|  1024|
|BrownTrout Publis...|   993|
|  Anonymous (Author)|   982|
|Hal Leonard Corp....|   940|
|    Fodor's (Author)|   905|
|Stephen King (Aut...|   904|
|Roger Priddy (Aut...|   902|
|Thomas Nelson (Au...|   813|
|William Shakespea...|   738|
|Rand McNally (Aut...|   707|
| Scholastic (Author)|   688|
|National 

Sample

In [0]:
from pyspark.sql.functions import col, regexp_extract, when
from pyspark.sql.types import DoubleType

In [0]:
df_with_price_cleaned = df_items_transformed.withColumn(
    "price_cleaned", 
    regexp_extract(col("price"), r"([\d\.]+)", 1) 
)
df_items_silver = df_with_price_cleaned \
    .withColumn("price_final",               
                when(col("price_cleaned") == "", None) 
                .otherwise(col("price_cleaned").cast(DoubleType()))
               ) \
    .withColumn("store_final",               
                when(col("store") == "n/a (Author)", None)
                .otherwise(col("store"))
               ) \
    .withColumn("main_category_final",
                when(col("main_category") == "", None)
                .otherwise(col("main_category"))
               ) \
    .drop("price", "price_cleaned", "store", "main_category") \
    .withColumnRenamed("price_final", "price") \
    .withColumnRenamed("store_final", "store") \
    .withColumnRenamed("main_category_final", "main_category") \
    .filter(col("parent_asin").isNotNull()) \
    .filter(col("title").isNotNull()) \
    .dropDuplicates(["parent_asin"])

In [0]:
print(f"Count row after drop duplicate asin: {df_items_silver.count()}")

Count row after drop duplicate asin: 4448180


In [0]:
df_items_silver.write \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("`bigdata-and-bi`.silver.items_clean")

In [0]:
print("--- Hiển thị 10 dòng kết quả (items_clean): ---")
display(df_items_silver.limit(10))

--- Hiển thị 10 dòng kết quả (items_clean): ---


title average_rating rating_number features description images videos categories details parent_asin bought_together price store main_category The Devil Can Ride: The World's Best Motorcycle Writing 4.1 9 List(Authors such as Hunter Thompson, Robert Pirsig, and Mark Singer have written about the motorcycle, that icon for outlaws, rebels, thieves, and beat poets. This collection of motorcycle tales features the best of the vast collection of motorcycle writing created since old Gottlieb Daimler first bolted a crude internal-combustion engine to his wooden two-wheeled Einspur in 1876. In addition to essays from Thompson and Pirsig, The Devil Can Ride features works by Peter Egan, T.E. Lawrence, James Stevenson, Jamie Elvidge, John Hall, and Kevin Cameron.) List(Review, “This is a book that can be opened to almost any page and deliver the kind of inspiration that fuels our passion for the people and machines in motorcycling.” –, Cycle World, October 2010, About the Author, Edited by magazine journalist and author Lee Klancher.  Klancher’s work has appeared in, Motorcyclist, Dirt Rider, Motorcycle Cruiser, ATV Rider, , and, Motorcycle Escape, magazines. Motorbooks published his most recent work,, Motorcycle Dream Garages, in 2009. Klancher has been riding, racing, and crashing motorcycles since he was 11 years old, and reading since he was three. He lives in Austin, TX.) List(Map(large -> https://m.media-amazon.com/images/I/51om0aJQzPL._SX339_BO1,204,203,200_.jpg, variant -> MAIN)) List() List(Books, Engineering & Transportation, Automotive) Map(Dimensions -> 6 x 1.06 x 9 inches, Language -> English, Item Weight -> 10.4 ounces, Hardcover -> 304 pages, ISBN 13 -> 978-0760334775, Publisher -> Motorbooks; First Edition (June 12, 2010), ISBN 10 -> 0760334773) 0760334773 null 30.0 Lee Klancher (Author), Kevin Cameron (Contributor), Jack Lewis (Contributor), Hunter S. Thompson (Contributor), Robert Pirsig (Contributor) Books Blessings 4.2 172 List(At thirty-six, Jennie Rakowsky's dreams were  coming true. She was about to marry a wonderful man,  her career as a lawyer was skyrocketing, and she  had never been more beautiful. And then the secret  she had hidden for nineteen years threatened to shatter it all.) List(Review, "Entertaining !"—, Washington Post, "Belva Plain's mixture of romance, suspense, and deeply felt familial conflicts should leave her fans well entertained."—, Publishers Weekly, About the Author, Belva Plain lives in northern New Jersey. She is the author of the bestselling novels, Evergreen, ,, Random Winds, ,, Eden Burning, ,, Crescent City, ,, The Golden Cup, ,, Tapestry, ,, Blessings, ,, Harvest, ,, Treasures, ,, Whispers, ,, Daybreak, ,, The Carousel, ,, Promises, ,, Secrecy, ,, Homecoming, ,, Legacy of Silence, , and, Fortune's Hand, ., Excerpt. © Reprinted by permission. All rights reserved., Chapter One, The day on which the sky cracked open over Jennie's head had begun as gladly as any other day in that wonderful year. It had been the best year of her life until then.At noon she had been standing with Jay on the lip of the hill that overlooked the wild land called, by the town to which it belonged, the Green Marsh. It was one of those Indian summer intervals, when, after two weeks of rain and premature gray cold, everything suddenly burns again; the distant air burns blue and the near oaks flare red; in the marsh, cattails and spreading juniper glisten darkly after the night's rain. Canada geese come streaming, honking their long way to the south; and ducks, with a great flapping racket, splash into the pond."You see, it's not all marsh," Jay explained. "There's meadow and forest at the other end. Over a thousand acres, all wild. Been here for Lord knows how many thousand of years, just as you see it, untouched. We're trying to get the state to take it over as part of the wilderness system. That way it'll be safe forever. But we've got to hurry before the New York builders put their bid through.""Do you suppos

### Review: bronze to silver

clean reviews table

In [0]:
df_reviews_transformed = spark.table("`bigdata-and-bi`.bronze.reviews_raw_5") \
    .withColumn("json_string", col("value").cast("string")) \
    .filter(col("json_string").isNotNull()) \
    .withColumn("parsed_json", from_json(col("json_string"), review_schema)) \
    .select("parsed_json.*") \
    .withColumn("reviewTimestamp",
                to_timestamp(col("timestamp"))) \
    .drop("timestamp")

print("Done.")

Done.


In [0]:
print(f"Tổng số dòng: {df_reviews_transformed.count()}")
print(f"Tổng số cột: {len(df_reviews_transformed.columns)}")

Tổng số dòng: 8787688
Tổng số cột: 10


In [0]:
df_reviews_transformed.groupBy("rating").count().orderBy("rating").show(50)

+------+-------+
|rating|  count|
+------+-------+
|   1.0| 244672|
|   2.0| 328591|
|   3.0| 770638|
|   4.0|1824057|
|   5.0|5619730|
+------+-------+



In [0]:
df_reviews_transformed.groupBy("helpful_vote").count().orderBy("helpful_vote").show(50)

+------------+-------+
|helpful_vote|  count|
+------------+-------+
|          -5|      1|
|           0|5493319|
|           1|1393481|
|           2| 560119|
|           3| 310919|
|           4| 197502|
|           5| 138623|
|           6| 101752|
|           7|  78009|
|           8|  61470|
|           9|  50227|
|          10|  41446|
|          11|  34766|
|          12|  29421|
|          13|  24990|
|          14|  21893|
|          15|  19069|
|          16|  16715|
|          17|  14925|
|          18|  13259|
|          19|  11797|
|          20|  10759|
|          21|   9607|
|          22|   8675|
|          23|   7766|
|          24|   7342|
|          25|   6622|
|          26|   6269|
|          27|   5592|
|          28|   5347|
|          29|   4729|
|          30|   4547|
|          31|   4233|
|          32|   3947|
|          33|   3555|
|          34|   3396|
|          35|   3121|
|          36|   3007|
|          37|   2860|
|          38|   2616|
|          

In [0]:
print("Kiểm tra số lượng NULL trong tất cả các cột của 'reviews':")
null_counts_expr = [
    f"count(case when {col_name} is null then 1 end) as null_{col_name}"
    for col_name in df_reviews_transformed.columns
]
display(df_reviews_transformed.selectExpr(*null_counts_expr))
empty_title_count = df_reviews_transformed.filter(col("title") == "").count()

Kiểm tra số lượng NULL trong tất cả các cột của 'reviews':


null_rating,null_title,null_text,null_images,null_asin,null_parent_asin,null_user_id,null_helpful_vote,null_verified_purchase,null_reviewTimestamp
0,0,0,0,0,0,0,0,0,0


In [0]:
#clean

df_reviews_silver = df_reviews_transformed \
    .filter(col("asin").isNotNull()) \
    .filter(col("user_id").isNotNull()) \
    .filter(col("rating").isNotNull()) \
    .filter(col("rating") >= 1.0) \
    .filter(col("rating") <= 5.0) \
    .dropDuplicates(["asin", "user_id", "reviewTimestamp"])

In [0]:
print(f"Tổng số dòng: {df_reviews_silver.count()}")
print(f"Tổng số cột: {len(df_reviews_silver.columns)}")

Tổng số dòng: 8719087
Tổng số cột: 10


In [0]:
df_reviews_silver.write \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("`bigdata-and-bi`.silver.reviews_clean_5_2")


In [0]:
print("--- Hiển thị 10 dòng kết quả (reviews_clean): ---")
display(df_reviews_silver.limit(10))

--- Hiển thị 10 dòng kết quả (reviews_clean): ---


rating,title,text,images,asin,parent_asin,user_id,helpful_vote,verified_purchase,reviewTimestamp
4.0,Book,Great read,List(),1558174850,1558174850,AHGPJ3OST45LWUERQAXMJNXEHOLA,0,true,2016-03-26T20:53:33.000Z
5.0,Mystifying!,"My oh My, I could hardly put it down - fun, fun, fun and extremely interesting too! Mystifying, educational, kept me on the edge of my seat from cover to cover.I enjoyed this book to the fullest. Another great book from Dan Brown!",List(),0385504225,0385504225,AFUPDCITID4PSZ7EJOJB4V2AB5IA,0,true,2010-06-20T17:20:11.000Z
5.0,Nutrition,"Gary Nulls is an excellent nutrition guru. I have several of his works, and I just recently purchase several of his DVDs. Excellent information and very helpful in solving many health related problems--you are what you eat!",List(),0451210506,0451210506,AEDAZOWHA4MMSQCIKNAUQVU5VZPQ,0,false,2010-08-09T22:54:34.000Z
5.0,Outstanding introduction to freestyle wrestling,"Most guides to combat sports are hampered by poor quality photography. This short, good-value book is an exception. The large, full-color photos show clearly how to execute moves and the text contains lots of personal testimonies from experienced wrestlers past and present.",List(),1402701071,1402701071,AHNDQE4EVGNLLGDUFW6NTDBEQCYA,3,false,2004-12-09T17:17:30.000Z
4.0,"Very interesting and innovative, but perhaps a bit overrated","I'm sure I'm courting fanboy wrath here, but I finally got around to reading ""Watchmen"" and I gotta tell ya... it ain't that great.Oh, believe me, it's helluva a comic book (""graphic novel"" is a pseduo-intellectualized marketing term). And as a work of imaginative fiction and innovative storytelling, it is impressive. I quite enjoyed reading it. I found it very interesting at several points, and even intellectually challenging (Alan Moore does love big words and literary references), but I just can't bring myself to think of it as some towering literary accomplishment for the ages. I suppose I'm just reacting to some of the slavering hyperbole heaped on in it by the geek squad (and I'm a proud member of that nerdy fraternity, mind you). Some times, things can be victims of their own hype. I think that's the case here.So go ahead and read ""Watchmen"" for yourself, if you're at all interested. It's certainly worth your time. If you approach it with a open mind and take it for what it is (a really excellent comic book), you'll be rewarded with a compelling text-and-visual story rich enough to appreciate on multiple levels. Given how much dreadful junk is out there in every medium, ""Watchmen"" really does stand pretty tall. I definitely tip my hat to it. I'm not going to genuflect at its altar though, fanboy piety be damned.[...]",List(),0930289234,0930289234,AFNO5MWQJ4R67RPXFCBKQLMMUV2A,3,true,2009-07-16T17:34:56.000Z
5.0,Expertly written,One of the best books ever written about air combat. You feel like you're one of the Jolly Rogers yourself !!,List(),0517570750,0517570750,AHFBBXF42H32I7EJDYKRG7AKMINQ,2,false,2001-11-01T11:42:30.000Z
3.0,Three Stars,Good premise but not great execution.,List(),039953671X,039953671X,AEZVDLZGCR3IHRXLZLZEXZ2BRVWA,0,true,2017-05-15T16:38:13.000Z
5.0,Worth reading,"Better to read than most ""self help"" books. It is not a ""how-to"", but it teaches you a lot about yourself. Put away your prejudices before you start reading.",List(),0969675534,0969675534,AGRURHMGC5OCDUI2JVHNFHLBHBSQ,0,false,2019-08-08T04:53:11.350Z
5.0,Great series.,"My hubby reads this type of book series, he really likes this author.",List(),1647349826,1647349826,AEDDVNHVPTTQ554AFPYDI5EJ34AQ,0,true,2022-05-17T18:17:03.274Z
4.0,Great book,Very informative.,List(),1482596148,1482596148,AFNAUKAGN4VGIYEB4GM7GAFDCHFQ,0,true,2018-11-22T01:26:29.776Z
